# LangChain - Assistant

### ![](../img/installation-ico.png) Installation

In [ ]:
pip install langchain langchain-community wikipedia faiss-cpu chromadb

### ![](../img/package-ico.png) Imports

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# import requests
# from bs4 import BeautifulSoup

import sys
import time
import pickle
from datetime import datetime

# from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.chat_models import ChatOllama
from langchain.vectorstores.chroma import Chroma
from langchain.schema import AIMessage, HumanMessage, SystemMessage

### ![](../img/parametrage-ico.png) Paramétrages

In [ ]:
global embeddings_flag
embeddings_flag = False

model = "mistral"
chat = ChatOllama(temperature=0, model=model)
messages = [SystemMessage(content="Toute réponse devra être faîtes en Français.")]
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        lang="fr", top_k_results=1, doc_content_chars_max=10000
    )
)
# wiki.validate_environment({"lang":"fr"})
# wikipedia.set_lang("fr")

In [ ]:
# Définition de l'appel à wikipedia
def get_wiki(search):

    return wiki.run(search).split("\n\n")
    # Issue with auto_suggest
    summary = wikipedia.summary(title=search, sentences=5, auto_suggest=False)

    # create URL based on user input and language
    url = f"https://fr.wikipedia.org/wiki/{search}"

    # send GET request to URL and parse HTML content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # extract main content of page
    content_div = soup.find(id="mw-content-text")

    # extract all paragraphs of content
    paras = content_div.find_all("p")

    # concatenate paragraphs into full page content
    full_page_content = ""
    for para in paras:
        full_page_content += para.text

    # print the full page content
    return full_page_content, summary


In [ ]:
# Gestion de l'index vectoriel
def build_index(wiki_content):
    print("building index .....")
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_text(wiki_content)
    embeddings = OllamaEmbeddings(model=model, num_gpu=1)
    docsearch = Chroma.from_texts(texts, embeddings)
    # docsearch = FAISS.from_texts(texts, embeddings)
    # with open("./embeddings.pkl", "wb") as f:
    #     pickle.dump(docsearch, f)

    return embeddings, docsearch

In [ ]:
# Envoi de la requête au chatbot
def get_chatbot_response(user_query, index):
    docs = index.similarity_search(user_query)
    # docs = index.similarity_search(user_query, K=6)
    main_content = user_query + "\n\n"
    for doc in docs:
        main_content += doc.page_content + "\n\n"
    messages.append(HumanMessage(content=main_content))
    ai_response = chat(messages).content
    messages.pop()
    messages.append(HumanMessage(content=user_query))
    messages.append(AIMessage(content=ai_response))

    return ai_response


# Affichage des messages
def display_messages(all_messages):
    for msg in all_messages:
        if msg["user"] == "user":
            print(
                f"You ({msg['time']}): {msg['text']}",
            )
        else:
            print(f"IA-Bot ({msg['time']}): {msg['text']}")


# Envoi des messages au chatbot
def send_message(user_query, index, all_messages):
    if user_query:
        all_messages.append(
            {
                "user": "user",
                "time": datetime.now().strftime("%H:%M"),
                "text": user_query,
            }
        )
        bot_response = get_chatbot_response(user_query, index)
        all_messages.append(
            {
                "user": "bot",
                "time": datetime.now().strftime("%H:%M"),
                "text": bot_response,
            }
        )

### ![](../img/jouer-ico.png) Exécution(s)

In [ ]:
# Messages à transmettre
messages = [
    SystemMessage(
        content="Vous êtes un robot Question/Réponse et vous répondez à toutes les questions posées par l'utilisateur. Si vous ne connaissez pas la réponse, indiquez 'Désolé, je ne sais pas'."
    )
]
# Stockage de la discussion
all_messages = []

search = input("Qu'avez-vous en tête ?")
# search = "La ville de Nantes"

if len(search):
    wiki_content = get_wiki(search)

    if len(wiki_content):
        try:
            # Create input text box for user to send messages
            print(f"First doc: {wiki_content[0]}")
            user_query = input("You: ")
            # user_query = "Quel est le nombre d'habitant"

            if not embeddings_flag:
                embeddings, docsearch = build_index("\n\n".join(wiki_content))
                embeddings_flag = True
                # with open("./embeddings.pkl", 'rb') as f:
                #     index = pickle.load(f)
                index = docsearch

            if embeddings_flag:
                send_message(user_query, index, all_messages)
                display_messages(all_messages)

        except Exception as e:
            print(
                "ERROR: quelque chose ne va pas... veuillez réessayer", file=sys.stderr
            )
            print(e)